In [1]:
from __future__ import print_function
import torch

import sys, os
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True'
sys.path.insert(1, './lustre/scratch117/cellgen/team292/rh26/16_explant_sc_2D9_stars_combine')

In [2]:
# Seed for reproducibility
import numpy as np
import pandas as pd
import scanpy as sc
from typing import Tuple
import anndata

# scVI imports
import scvi
from scvi.dataset import AnnDatasetFromAnnData
from scvi.inference import UnsupervisedTrainer
from scvi.models.vae import VAE

torch.manual_seed(0)
np.random.seed(0)
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)



def MovePlots(plotpattern, subplotdir):
    os.system('mkdir -p '+str(sc.settings.figdir)+'/'+subplotdir)
    os.system('mv '+str(sc.settings.figdir)+'/*'+plotpattern+'** '+str(sc.settings.figdir)+'/'+subplotdir)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = './figures/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable

-----
anndata     0.7.6
scanpy      1.7.0
sinfo       0.3.1
-----
PIL                 8.3.0
absl                NA
anndata             0.7.6
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
cairo               1.20.1
certifi             2021.05.30
cffi                1.14.4
chardet             4.0.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
debugpy             1.3.0
decorator           5.0.9
defusedxml          0.7.1
dunamai             1.5.5
fsspec              2021.06.1
get_version         3.2
google              NA
h5py                3.3.0
idna                2.10
igraph              0.8.3
ipykernel           6.0.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
json5               NA
jsonschema          3.2.0
jupyter_ser

'/opt/conda/envs/scvi-singularity/bin/python'

### Read in dataset

In [3]:
adata = sc.read('S1_rna_counts_adata.h5ad')
adata.X.shape

(55409, 35220)

In [4]:
list(np.unique(adata.obs['stage'])) 
adata.obs['stage'].value_counts()

day2_INF    34372
day2_UI     21037
Name: stage, dtype: int64

In [5]:
# do some basic preprocessing
adata.layers["raw_counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata.copy()

normalizing counts per cell
    finished (0:00:00)


In [6]:
np.unique(adata.obs['donor_id'])

array(['Hrv107', 'Hrv124'], dtype=object)

In [7]:
adata.obs['sample']

Pla_HDBR11345029_AAACCCAAGACATCAA    Pla_HDBR11345029
Pla_HDBR11345029_AAACCCAAGCTTAAGA    Pla_HDBR11345029
Pla_HDBR11345029_AAACCCAAGGTTATAG    Pla_HDBR11345029
Pla_HDBR11345029_AAACCCACAGGTCTCG    Pla_HDBR11345029
Pla_HDBR11345029_AAACCCAGTTACACAC    Pla_HDBR11345029
                                           ...       
Pla_HDBR11345034_TTTGTTGGTGGAGAAA    Pla_HDBR11345034
Pla_HDBR11345034_TTTGTTGGTGTTTGCA    Pla_HDBR11345034
Pla_HDBR11345034_TTTGTTGGTTATGTGC    Pla_HDBR11345034
Pla_HDBR11345034_TTTGTTGTCCTGGGAC    Pla_HDBR11345034
Pla_HDBR11345034_TTTGTTGTCGTAGAGG    Pla_HDBR11345034
Name: sample, Length: 55409, dtype: category
Categories (4, object): ['Pla_HDBR11345029', 'Pla_HDBR11345030', 'Pla_HDBR11345033', 'Pla_HDBR11345034']

In [8]:
scvi.data.setup_anndata(
    adata,
    layer="raw_counts",
    batch_key='donor_id', #samples as a batch
    #donor as a covariate of the cells
    categorical_covariate_keys=['sample'] 
    #continuous_covariate_keys=[""]
)

INFO     Using batches from adata.obs["donor_id"]                                            


Using batches from adata.obs["donor_id"]


INFO     No label_key inputted, assuming all cells have same label                           


No label_key inputted, assuming all cells have same label


INFO     Using data from adata.layers["raw_counts"]                                          


...

Computing library size prior per batch


INFO     Successfully registered anndata object containing 55409 cells, 35220 vars, 2        
         batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates   
         and 0 extra continuous covariates.                                                  


Successfully registered anndata object containing 55409 cells, 35220 vars, 2 batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates and 0 extra continuous covariates.


INFO     Please do not further modify adata until model is trained.                          


Please do not further modify adata until model is trained.


In [9]:
models = {}
n_latent_values = [10,  30,  50]
#n_latent_values = [10]

for n_latent_value in n_latent_values:
    print('n_latent_value', n_latent_value)
    models[n_latent_value] = scvi.model.SCVI(adata, n_latent = n_latent_value)

n_latent_value 10
n_latent_value 30
n_latent_value 50


In [12]:
models[n_latent_value]

SCVI Model with the following params: 
n_hidden: 128, n_latent: 50, n_layers: 1, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: zinb, latent_distribution: normal
Training status: Not Trained

To print summary of associated AnnData, use: scvi.data.view_anndata_setup(model.adata)

## Save latent space

In [13]:
latent_representations = {}

for n_latent_value in n_latent_values:
    print('training model for n_latent_value:', n_latent_value)
    models[n_latent_value].train()
    
    latent_representations[n_latent_value] = models[n_latent_value].get_latent_representation()
    
    adata.obsm["X_scVI_n_latent_" + str(n_latent_value)] = latent_representations[n_latent_value]
    
    curr_df = pd.DataFrame(adata.obsm["X_scVI_n_latent_" + str(n_latent_value)])
    curr_df.to_csv('./sc_combi_scVI/20220414_sc_combi_obsm_with_scVI_latent_representation_n_' + str(n_latent_value) + '_corrected_by_donor.csv')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 10
Epoch 144/144: 100%|██████████████████████████████████████| 144/144 [23:28<00:00,  9.78s/it, loss=9.98e+03, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 30
Epoch 144/144: 100%|██████████████████████████████████████| 144/144 [23:34<00:00,  9.82s/it, loss=9.96e+03, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 50
Epoch 144/144: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 144/144 [23:32<00:00,  9.81s/it, loss=1e+04, v_num=1]
